In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score

# 📌 **1. Veri Ön İşleme Fonksiyonu**
def preprocess_data(file_path, is_train=True, scaler=None, encoder_dict=None):
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 🟢 1. Kategorik değişkenleri encode et
    categorical_cols = df.select_dtypes(include=['object']).columns
    if is_train:
        encoder_dict = {}
        for col in categorical_cols:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            encoder_dict[col] = le
    else:
        for col in categorical_cols:
            if col in encoder_dict:
                df[col] = df[col].map(lambda s: encoder_dict[col].transform([s])[0] if s in encoder_dict[col].classes_ else -1)

    print("✅ Kategorik değişkenler encode edildi!")

    # 🟢 2. Eksik değerleri doldur
    df.fillna(df.median(), inplace=True)
    print("✅ Eksik değerler dolduruldu!")

    # Hedef değişkeni ayır
    y = df["rainfall"] if "rainfall" in df.columns else None
    X = df.drop(columns=["rainfall"]) if "rainfall" in df.columns else df

    # 🟢 3. Sayısal değişkenleri ölçeklendir
    if is_train:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    else:
        X = scaler.transform(X)

    print("✅ Sayısal değişkenler ölçeklendirildi!")
    
    return X, y, scaler, encoder_dict

# 📌 **2. XGBoost Model Eğitme Fonksiyonu**
def train_xgboost(X, y):
    """
    XGBoost Modeli ile eğitimi gerçekleştirir.
    """
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = xgb.XGBClassifier(
        n_estimators=500,  # Daha yüksek ağaç sayısı
        learning_rate=0.01,  # Küçük adımlarla öğren
        max_depth=6,  # Derinlik
        colsample_bytree=0.8,  # Feature sampling
        subsample=0.8,  # Veri alt örnekleme
        eval_metric="auc",
        use_label_encoder=False
    )

    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=20, verbose=10)

    y_pred = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_pred)

    print(f"✅ XGBoost Modeli Eğitildi! ROC AUC: {auc_score:.4f}")
    
    return model

# 📌 **3. Test Verisi Üzerinde Tahmin Yapma Fonksiyonu**
def predict_and_save(model, test_file, output_file, scaler, encoder_dict):
    X_test, _, _, _ = preprocess_data(test_file, is_train=False, scaler=scaler, encoder_dict=encoder_dict)
    
    predictions = model.predict_proba(X_test)[:, 1]
    
    # Test seti ID'sini al
    test_df = pd.read_csv(test_file)
    test_ids = test_df["id"]

    # Kaggle formatında CSV oluştur
    submission = pd.DataFrame({"id": test_ids, "rainfall": predictions})
    submission.to_csv(output_file, index=False)
    
    print(f"✅ Tahminler {output_file} dosyasına kaydedildi!")

# 📌 **4. Çalıştırma Kodu**
train_file = "train.csv"
test_file = "test.csv"
output_file = "xgboost_submission.csv"

# 1️⃣ Veriyi işle
X_train, y_train, scaler, encoder_dict = preprocess_data(train_file, is_train=True)

# 2️⃣ Modeli eğit (XGBoost kullanıyoruz)
xgb_model = train_xgboost(X_train, y_train)

# 3️⃣ Test verisi üzerinde tahmin yap ve sonucu kaydet
predict_and_save(xgb_model, test_file, output_file, scaler, encoder_dict)
